# DREAM diagnostics

This notebook contains a set of visualizations for the DREAM detectors.
The main purpose of such visualizations are to provide quick diagnostics on the detector data,
which help determine if some of the detector components (wires, strips, modules...) are faulty or broken.

In [ ]:
import pandas as pd
import scipp as sc
import plopp as pp
import matplotlib.pyplot as plt
from ess.dream import diagnostics as diags
from ess import dream

## Load and reshape the data

We load a data set from a Geant4 simulation (stored as a `.csv` file).
After loading, we manipulate and reshape the data to obtain a data group that contains one entry per detector bank.
In each detector bank, the data is organised by `wire`, `strip`, `module`, `segment`, and `counter`.

It is assumed that Nexus files from the DREAM beamline will be loaded as such by Scippnexus,
and that the manipulations required below would not be needed for production datasets.

In [ ]:
filename = dream.data['data_dream_HF_mil_closed_alldets_1e9.csv.zip']
df = pd.read_table(filename)
ds = sc.compat.from_pandas(df)
del ds.coords['row']  # we have no use for this row index
# Some logic to parse units in the file header
for key in list(ds):
    name, *remainder = key.split(' ')
    ds[name] = ds.pop(key)
    ds[name].unit = remainder[0][1:-1] if remainder else None
ds['lambda'].unit = 'angstrom'
table = sc.DataArray(sc.ones(sizes=ds.sizes, unit='counts'))
for name in ds:
    table.coords[name] = ds[name].data
# Group pixels according to their bank identifier
mapping = {'sumo0': 0, 'sumo1': 1, 'sumo2': 2, 'sumo3': 3, 'mantle': 4, 'high_res': 5}
da = table.group('det')
dg = sc.DataGroup({key: da['det', val] for key, val in mapping.items()})
dg = sc.DataGroup({key: diags.to_logical_dims(da) for key, da in dg.items()})
for da in dg.values():
    da.coords['x'] = da.bins.coords.pop('x_pos').bins.mean()
    da.coords['y'] = da.bins.coords.pop('y_pos').bins.mean()
    da.coords['z'] = da.bins.coords.pop('z_pos').bins.mean()
dg

## Detector diagnostic plots

We now make a single large figure showing some of the most commonly used plots,
with one column per detector bank.

In [ ]:
fig, ax = plt.subplots(4, len(dg), figsize=(len(dg) * 3, 4 * 3))
for i, (name, da) in enumerate(dg.items()):
    da.bins.concat(list(set(da.dims) - {'module'})).hist().plot(ax=ax[0, i], title=name)
    diags.module_vs_segment(da).hist().plot(ax=ax[1, i])
    da.bins.concat(list(set(da.dims) - {'strip'})).hist().plot(ax=ax[2, i])
    diags.wire_vs_strip(da).hist().plot(ax=ax[3, i])
fig.subplots_adjust(wspace=0.5, hspace=0.5)

## Instrument view

We also show the 3D view of the instrument pixels.
The DREAM-specific instrument view is also capable of slicing the data along the `tof` dimension by using the argument `dim='tof'`
(use the `bins` argument to change how many bins will be used to histogram the `tof` dimension).

In [ ]:
# In order to keep the size of the doc pages down, we only keep 1 in 10 pixels
dg = sc.DataGroup({key: da['pixel', ::10] for key, da in dg.flatten(to='pixel').items()})

In [ ]:
dream.instrument_view(dg, dim='tof', bins=10, pixel_size=20)

Finally, it is also possible to look at a single bank using

In [ ]:
dream.instrument_view(dg['mantle'], pixel_size=20)